In [ ]:
import pandas as pd
import numpy as np
import os
import json
from a import MaxEntScorer
from metrics import compute_metrics, get_confusion_matrix

# --- CẤU HÌNH ---
DATA_DIR = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\data"
OUTPUT_DIR = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

scorer = MaxEntScorer()

def softmax(x):
    # Tránh tràn số (numerical stability)
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)

if __name__ == "__main__":
    RATIOS = ["1-1-1", "2-1-1", "4-1-1", "10-1-1", "100-1-1"]
    
    # CHIẾN THUẬT BIAS (Cộng thêm vào logit để ép nhãn)
    # Vì Score của MaxEntScan bản chất gần giống log-likelihood
    # Ta cộng thêm giá trị dương để ưu tiên lớp đó
    CLASS_BIAS = np.array([5.0, 1.0, 4.0]) # Ép mạnh vào lớp 2 (Acceptor)

    for ratio in RATIOS:
        input_file = os.path.join(DATA_DIR, f"maxent_input_{ratio}.csv")
        if not os.path.exists(input_file):
            print(f"❌ Không tìm thấy file: {input_file}")
            continue
            
        print(f"\n🚀 Đang chạy chế độ BIAS CỰC ĐOAN cho Ratio: {ratio}")
        df = pd.read_csv(input_file)
        
        # 1. Tính Scores thô
        s5_scores = df['seq_5'].apply(scorer.score5).values
        s3_scores = df['seq_3'].apply(scorer.score3).values
        
        # 2. Tạo ma trận điểm [L0, L1, L2]
        # Lớp 0 (Baseline) chúng ta đặt là điểm trung bình thấp (ví dụ -5.0) 
        # thay vì 0.0 để tránh nó lấn át các site yếu.
        combined_scores = np.stack([np.zeros(len(s5_scores)), s5_scores, s3_scores], axis=1)
        
        # 3. Áp dụng Bias (Cộng thay vì Nhân để tránh lỗi số âm)
        weighted_logits = combined_scores + CLASS_BIAS
        
        # 4. Dự đoán
        all_probs = softmax(weighted_logits)
        all_preds = np.argmax(weighted_logits, axis=1)
        
        # 5. Tính Metrics
        y_true = df['label'].values
        # Đảm bảo hàm compute_metrics của bạn nhận diện đúng thứ tự nhãn
        metrics_res = compute_metrics(y_true, all_preds, probs=all_probs, k=2)
        metrics_res['confusion_matrix'] = get_confusion_matrix(y_true, all_preds).tolist()
        
        # Thêm thông tin về thiết lập cực đoan vào kết quả
        metrics_res['meta'] = {"bias_applied": CLASS_BIAS.tolist(), "ratio": ratio}

        output_json = os.path.join(OUTPUT_DIR, f"results_{ratio}.json")
        with open(output_json, "w") as f:
            json.dump(metrics_res, f, indent=4)
            
        print(f"✅ Đã hoàn tất xử lý. Kết quả lưu tại: {output_json}")


🚀 Đang chạy chế độ BIAS CỰC ĐOAN cho Ratio: 1-1-1
✅ Đã hoàn tất xử lý. Kết quả lưu tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\results\results_1-1-1.json

🚀 Đang chạy chế độ BIAS CỰC ĐOAN cho Ratio: 2-1-1
✅ Đã hoàn tất xử lý. Kết quả lưu tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\results\results_2-1-1.json

🚀 Đang chạy chế độ BIAS CỰC ĐOAN cho Ratio: 4-1-1
✅ Đã hoàn tất xử lý. Kết quả lưu tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\results\results_4-1-1.json

🚀 Đang chạy chế độ BIAS CỰC ĐOAN cho Ratio: 10-1-1
✅ Đã hoàn tất xử lý. Kết quả lưu tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\results\results_10-1-1.json
